In [1]:
# 코드 7-29 라이브러리 호출
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
from torch.nn import Parameter  # 파라미터 목록을 갖고 있는 라이브러리(패키지)
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import math  # 수학과 관련되어 다양한 함수들과 상수들이 정의되어 있는 라이브러리

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cuda = (
    True if torch.cuda.is_available() else False
)  # GPU 사용에 필요(GPU는 8장에서 자세히 살펴볼 예정이지만 미리 눈으로 익혀 두세요)

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor  # GPU 사용에 필요

torch.manual_seed(125)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(125)

In [2]:
# 코드 7-30 데이터 전처리
import torchvision.transforms as transforms

mnist_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            (0.5,), (1.0,)
        ),  # 평균을 0.5, 표준편차를 1.0으로 데이터 정규화(데이터 분포를 조정)
    ]
)

In [3]:
# 코드 7-31 데이터셋 내려받기
from torchvision.datasets import MNIST

download_root = "../chap07/MNIST_DATASET"  # MNIST를 내려받을 경로

train_dataset = MNIST(
    download_root, transform=mnist_transform, train=True, download=True
)
valid_dataset = MNIST(
    download_root, transform=mnist_transform, train=False, download=True
)
test_dataset = MNIST(
    download_root, transform=mnist_transform, train=False, download=True
)

Failed to download (trying next):
HTTP Error 403: Forbidden



  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../chap07/MNIST_DATASET/MNIST/raw/train-images-idx3-ubyte.gz to ../chap07/MNIST_DATASET/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../chap07/MNIST_DATASET/MNIST/raw/train-labels-idx1-ubyte.gz to ../chap07/MNIST_DATASET/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../chap07/MNIST_DATASET/MNIST/raw/t10k-images-idx3-ubyte.gz to ../chap07/MNIST_DATASET/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../chap07/MNIST_DATASET/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../chap07/MNIST_DATASET/MNIST/raw



/home/halozhan/딥러닝 파이토치 교과서/.venv/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
# 코드 7-32 데이터셋을 메모리로 가져오기
batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [5]:
# 코드 7-33 변수 값 지정
batch_size = 100
n_iters = 6000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [6]:
# 코드 7-34 LSTM 셀 네트워크 구축
class LSTMCell(
    nn.Module
):  # LSTM 셀에 대한 더 자세한 설명을 원한다면 http://www.bioinf.jku.at/publications/older/2604.pdf 논문을 참고하세요.
    def __init__(self, input_size, hidden_size, bias=True):
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 4 * hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 4 * hidden_size, bias=bias)
        self.reset_parameters()

    def reset_parameters(self):  # 모델의 파라미터 초기화
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std)  # ②

    def forward(self, x, hidden):
        hx, cx = hidden
        x = x.view(-1, x.size(1))

        gates = self.x2h(x) + self.h2h(hx)
        gates = gates.squeeze()
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

        ingate = F.sigmoid(ingate)  # 입력 게이트에 시그모이드 활성화 함수 적용
        forgetgate = F.sigmoid(forgetgate)  # 망각 게이트에 시그모이드 활성화 함수 적용
        cellgate = F.tanh(cellgate)  # 셀 게이트에 탄젠트 활성화 함수 적용
        outgate = F.sigmoid(outgate)  # 출력 게이트에 시그모이드 활성화 함수 적용

        cy = torch.mul(cx, forgetgate) + torch.mul(ingate, cellgate)
        hy = torch.mul(outgate, F.tanh(cy))
        return (hy, cy)

In [7]:
# 코드 7-35 LSTM 셀의 전반적인 네트워크
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim  # 은닉층의 뉴런/유닛 개수

        self.layer_dim = layer_dim
        self.lstm = LSTMCell(input_dim, hidden_dim, layer_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        if torch.cuda.is_available():  # GPU 사용 유무 확인
            h0 = Variable(
                torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda()
            )  # (은닉층의 계층 개수, 배치 크기, 은닉층의 뉴런 개수) 형태를 갖는 은닉 상태를 0으로 초기화
        else:
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))

        if torch.cuda.is_available():  # GPU 사용 유무 확인
            c0 = Variable(
                torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda()
            )  # (은닉층의 계층 개수, 배치 크기, 은닉층의 뉴런 개수) 형태를 갖는 셀 상태를 0으로 초기화
        else:
            c0 = Variable(torch.zeros(self.layer_dim, x.size(0), hidden_dim))

        outs = []
        cn = c0[
            0, :, :
        ]  # (은닉층의 계층 개수, 배치 크기, 은닉층의 뉴런 개수) 크기를 갖는 셀 상태에 대한 텐서
        hn = h0[
            0, :, :
        ]  # (은닉층의 계층 개수, 배치 크기, 은닉층의 뉴런 개수) 크기를 갖는 은닉 상태에 대한 텐서

        for seq in range(x.size(1)):  # LSTM 셀 계층을 반복하여 쌓아 올립니다.
            hn, cn = self.lstm(
                x[:, seq, :], (hn, cn)
            )  # 은닉 상태(hn)와 셀 상태를 LSTMCell에 적용한 결과를 또다시 hn, cn에 저장
            outs.append(hn)

        out = outs[-1].squeeze()
        out = self.fc(out)
        return out

In [8]:
# 코드 7-36 옵티마이저와 손실 함수 지정
input_dim = 28
hidden_dim = 128
layer_dim = 1
output_dim = 10

model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
if torch.cuda.is_available():  # GPU 사용 유무 확인
    model.cuda()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [9]:
# 코드 7-37 모델 학습 및 성능 확인
seq_dim = 28
loss_list = []
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(
        train_loader
    ):  # 훈련 데이터셋을 이용한 모델 학습
        if torch.cuda.is_available():  # GPU 사용 유무 확인
            images = Variable(images.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:  # GPU를 사용하지 않기 때문에 else 구문이 실행
            images = Variable(images.view(-1, seq_dim, input_dim))
            labels = Variable(labels)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)  # 손실 함수를 이용하여 오차 계산

        if torch.cuda.is_available():
            loss.cuda()

        loss.backward()
        optimizer.step()  # 파라미터 업데이트
        loss_list.append(loss.item())
        iter += 1

        if iter % 500 == 0:  # 정확도(accuracy) 계산
            correct = 0
            total = 0
            for images, labels in valid_loader:  # 검증 데이터셋을 이용한 모델 성능 검증

                if torch.cuda.is_available():
                    images = Variable(images.view(-1, seq_dim, input_dim).cuda())
                else:
                    images = Variable(images.view(-1, seq_dim, input_dim))

                outputs = model(images)
                _, predicted = torch.max(
                    outputs.data, 1
                )  # 모델을 통과한 결과의 최댓값으로부터 예측 결과 가져오기

                total += labels.size(0)  # 총 레이블 수
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()

            accuracy = 100 * correct / total
            print(
                "Iteration: {}. Loss: {}. Accuracy: {}".format(
                    iter, loss.item(), accuracy
                )
            )

/home/halozhan/딥러닝 파이토치 교과서/.venv/lib/python3.8/site-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/halozhan/딥러닝 파이토치 교과서/.venv/lib/python3.8/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Iteration: 500. Loss: 2.237457275390625. Accuracy: 21.420000076293945
Iteration: 1000. Loss: 0.8789322376251221. Accuracy: 74.98999786376953
Iteration: 1500. Loss: 0.4282386898994446. Accuracy: 89.05000305175781
Iteration: 2000. Loss: 0.253286749124527. Accuracy: 92.22000122070312
Iteration: 2500. Loss: 0.05650099366903305. Accuracy: 95.80999755859375
Iteration: 3000. Loss: 0.06671759486198425. Accuracy: 96.1500015258789
Iteration: 3500. Loss: 0.12194900959730148. Accuracy: 95.4800033569336
Iteration: 4000. Loss: 0.03753276541829109. Accuracy: 96.95999908447266
Iteration: 4500. Loss: 0.05447082221508026. Accuracy: 97.29000091552734
Iteration: 5000. Loss: 0.08735618740320206. Accuracy: 96.95999908447266
Iteration: 5500. Loss: 0.16384145617485046. Accuracy: 97.37000274658203
Iteration: 6000. Loss: 0.007440885063260794. Accuracy: 97.70999908447266
Iteration: 6500. Loss: 0.03292839229106903. Accuracy: 97.80000305175781
Iteration: 7000. Loss: 0.025537937879562378. Accuracy: 97.6399993896484

In [12]:
# 코드 7-38 테스트 데이터셋을 이용한 모델 예측 성능 확인
def evaluate(model, val_iter):
    corrects, total, total_loss = 0, 0, 0
    model.eval()
    for images, labels in val_iter:
        if torch.cuda.is_available():  # GPU 사용 유무 확인
            images = Variable(images.view(-1, seq_dim, input_dim).cuda())
        else:
            images = Variable(images.view(-1, seq_dim, input_dim)).to(device)

        logit = model(images).to(device)
        labels = labels.to(device)
        loss = F.cross_entropy(
            logit, labels, reduction="sum"
        )  # reduction=‘sum’을 지정했기 때문에 모든 오차를 더합니다.
        _, predicted = torch.max(
            logit.data, 1
        )  # logit.data 텐서에서 최댓값의 인덱스(index)를 반환
        total += labels.size(0)
        total_loss += loss.item()
        corrects += (predicted == labels).sum()

    avg_loss = total_loss / len(val_iter.dataset)
    avg_accuracy = corrects / total
    return avg_loss, avg_accuracy

In [13]:
# 코드 7-39 모델 예측 성능 확인
test_loss, test_acc = evaluate(model, test_loader)
print("Test Loss: %5.2f | Test Accuracy: %5.2f" % (test_loss, test_acc))

/home/halozhan/딥러닝 파이토치 교과서/.venv/lib/python3.8/site-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/halozhan/딥러닝 파이토치 교과서/.venv/lib/python3.8/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Test Loss:  0.06 | Test Accuracy:  0.98
